# Aplicar el análisis de clases latentes

## Configurar



In [20]:
#eliminar archivos previos y limpiar la memoria del entorno
rm(list=ls());gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2986718,159.6,5616068,300.0,5616068,300.0
Vcells,6000961,45.8,21284564,162.4,21284564,162.4


In [21]:
#cargar datos
load("/content/palestine_25.RData")

#Definir el repositorio sobre el que instalar los paquetes desde Chile
#options(repos=structure(c(CRAN="https://cran.dcc.uchile.cl/")))

url <- "https://www.dropbox.com/scl/fi/3o58dwzew97q2r69r8vi3/palestine_25.RData?rlkey=6ur8jbxgl12h66k49e2e93325&st=yzcaoo4i&dl=1"
download.file(url, destfile = "palestine_25.RData", mode = "wb")

# Cargar el archivo
load("palestine_25.RData")


options(install.packages.check.source = "yes") # Chequea la fuente de los paquetes


if(!require(pacman)){install.packages("pacman");require(pacman)}

pacman::p_unlock(lib.loc = .libPaths()) #para no tener problemas reinstalando paquetes

#obliga a desplegarlo en la versión de R 4.4.0
if(Sys.info()["sysname"]=="Windows"){
if (getRversion() != "4.4.0") { stop("Requiere versión de R 4.4.0. Actual: ", getRversion()) }
}

# Cargar o instalar paquetes necesarios
#pacman::p_load(
#  showtext,   # Para cambiar la fuente de las letras
#  ggplot2,    # Para elaborar gráficos
#  plotly,     # Para elaborar gráficos interactivos
#  magick,     # Para mostrar imágenes
#  knitr,      # Para hacer tablas e interactuar con informes
#  kableExtra, # Tablas más bonitas
#  tidyverse,  # Para manipular bases de datos
#  rio,        # Para importar y exportar bases de datos en distintos formatos
#  psych,      # Para explorar variables
#  parallel,   # Para paralelizar los procesos en la CPU
#  doParallel, # Para paralelizar los procesos en la CPU
#  glca,       # Para llevar a cabo análisis de clases latentes
#  DiagrammeR, # Para generar gráficos esquemáticos
#  DiagrammeRsvg, # Para exportar gráficos
#  rsvg,       # Para transformar gráficos en formato .svg
#  htmlwidgets, # Para visualizarlos en una presentación
#  janitor,    # Permite limpiar bases de datos, entre otras funciones
#  tableone,
#  install = TRUE # Instala los paquetes si no están presentes
#)

url <- "https://www.dropbox.com/scl/fi/a76o70v6fjyy7pzgsu1vr/paquetes.tar.gz?rlkey=17ms7rx434wyz60y4ty3isovf&st=uw63uatn&dl=1"
download.file(url, destfile = "paquetes.tar.gz", mode = "wb")

# Paso 2: Descomprimir el archivo
untar("paquetes.tar.gz")

# Paso 3: Instalar todos los paquetes desde los .tar.gz
paquetes <- list.files("paquetes_tar", pattern = "\\.tar\\.gz$", full.names = TRUE)
for (pkg in paquetes) {
  install.packages(pkg, repos = NULL, type = "source")
}

#para comparaciones
if(!require(chisq.posthoc.test)){devtools::install_github("ebbertd/chisq.posthoc.test")}


No 00LOCK detected in:
 /usr/local/lib/R/site-library
No 00LOCK detected in:
 /usr/lib/R/site-library
No 00LOCK detected in:
 /usr/lib/R/library

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in inst

## Describimos los datos de interés


In [22]:
variables_interes <- c("QKUW40_1", "QKUW40_2", "QKUW40_3", "QKUW40_4",
                       "QKUW40_5", "QKUW40_90", "QKUW40_97", "QKUW40_98",
                       "QKUW40_99", "Q1002", "Q1005")


# Crear un subconjunto de las variables de interés, actualizado
variables_interes_act <-
c("Apoyo1. Boicot a empresas que apoyan a Israel", "Apoyo2. Seguimiento continuo de noticias relacionadas con la guerra",
"Apoyo3. Participación en actividades públicas de solidaridad",
"Apoyo4. Difusión de mensajes de solidaridad en redes sociales",
"Apoyo5. Donaciones monetarias a los residentes de Gaza", "Apoyo90. Otro – Por favor, especifique:",
"Apoyo97. No he tomado ninguna de estas acciones", "Apoyo98. No sabe",
"Apoyo99. Se niega a responder", "Sexo", "Estatus ocupacional")

# Generar la tabla descriptiva
tabla_descriptiva <- CreateTableOne(vars = variables_interes_act,
      data = arabebarometro_selected%>%
         dplyr::mutate(across(
           starts_with("QKUW40_"), ~dplyr::case_when(
             .==0~"No", .==1~"Sí", T~NA_character_
           )))%>%
        dplyr::mutate(Q1005= dplyr::case_when(
    Q1005 == "1" ~ "Empleado",
    Q1005 == "2" ~ "Auto-empleado",
    Q1005 == "3" ~ "Retirado",
    Q1005 == "4" ~ "Dueñ@ de casa",
    Q1005 == "5" ~ "Estudiante",
    Q1005 == "6" ~ "Desempleado/buscando",
    Q1005 == "99" ~ "No responde",
    TRUE ~ NA_character_
  ))%>%
    dplyr::mutate(Q1002= dplyr::case_when(
      Q1002== "1"~ "Hombre",
      Q1002== "2"~ "Mujer",
      T~ NA_character_)) %>%
  dplyr::rename(
    "Apoyo1. Boicot a empresas que apoyan a Israel" = "QKUW40_1",
    "Apoyo2. Seguimiento continuo de noticias relacionadas con la guerra" = "QKUW40_2",
    "Apoyo3. Participación en actividades públicas de solidaridad" = "QKUW40_3",
    "Apoyo4. Difusión de mensajes de solidaridad en redes sociales" = "QKUW40_4",
    "Apoyo5. Donaciones monetarias a los residentes de Gaza" = "QKUW40_5",
    "Apoyo90. Otro – Por favor, especifique:" = "QKUW40_90",
    "Apoyo97. No he tomado ninguna de estas acciones" = "QKUW40_97",
    "Apoyo98. No sabe" = "QKUW40_98",
    "Apoyo99. Se niega a responder" = "QKUW40_99",
    "Sexo" = "Q1002",
    "Estatus ocupacional" = "Q1005"),
  factorVars = variables_interes_act)

# Mostrar la tabla en consola
tabla_df <- as.data.frame(print(tabla_descriptiva, showAllLevels = TRUE, missing = TRUE))
tabla_df <-cbind.data.frame(var= gsub("\\s+", " ",gsub("\\.", " ", rownames(tabla_df))), tabla_df)
rownames(tabla_df)<-NULL
# tabla_df$var <- sub("\\.\\.", ": ", tabla_df$var)
# tabla_df$var <- sub("\\.\\.\\.\\.", "", tabla_df$var)
# tabla_df$var <- sub("\\.", " ", tabla_df$var)
# tabla_df$var <- sub("\\.", " ", tabla_df$var)
# tabla_df$var <- sub("\\.", " ", tabla_df$var)
# tabla_df$var <- sub("\\.$", "", tabla_df$var)
tabla_df$var <- ifelse(grepl("^X", tabla_df$var), "", tabla_df$var)

                                                                         
                                                                          level               
  n                                                                                           
  Apoyo1. Boicot a empresas que apoyan a Israel (%)                       No                  
                                                                          Sí                  
  Apoyo2. Seguimiento continuo de noticias relacionadas con la guerra (%) No                  
                                                                          Sí                  
  Apoyo3. Participación en actividades públicas de solidaridad (%)        No                  
                                                                          Sí                  
  Apoyo4. Difusión de mensajes de solidaridad en redes sociales (%)       No                  
                                                                       

In [23]:
tabla_df%>%
  knitr::kable("markdown", caption="Descriptivos", col.names=c("Variable", "Categoría", "Total", "Perdidos"))#%>%
  #kableExtra::kable_styling(bootstrap_options = c("striped", "hover"),font_size = 12) %>%
  #kableExtra::scroll_box(width = "100%", height = "575px")



Table: Descriptivos

|Variable                                                           |Categoría            |Total       |Perdidos |
|:------------------------------------------------------------------|:--------------------|:-----------|:--------|
|n                                                                  |                     |1210        |         |
|Apoyo1 Boicot a empresas que apoyan a Israel                       |No                   |200 (16.5)  |0.0      |
|                                                                   |Sí                   |1010 (83.5) |         |
|Apoyo2 Seguimiento continuo de noticias relacionadas con la guerra |No                   |598 (49.4)  |0.0      |
|                                                                   |Sí                   |612 (50.6)  |         |
|Apoyo3 Participación en actividades públicas de solidaridad        |No                   |950 (78.5)  |0.0      |
|                                                        

Generamos el análisis de clases latentes utilizando distintas soluciones, que van desde las 02 a las 10 clases.


In [27]:
vector_items<- c("QKUW40_1", "QKUW40_2", "QKUW40_3", "QKUW40_4", "QKUW40_5", "QKUW40_90", "QKUW40_97", "QKUW40_98", "QKUW40_99")
colSums(is.na(arabebarometro_selected[,vector_items]))


QKUW40_1  QKUW40_2  QKUW40_3  QKUW40_4  QKUW40_5 QKUW40_90 QKUW40_97 QKUW40_98 
        0         0         0         0         0         0         0         0 
QKUW40_99 
        0

In [ ]:
#Definimos el modelo: ítems manifiestos son explicados por una variable latente
f_lca<- item(QKUW40_1, QKUW40_2, QKUW40_3, QKUW40_4, QKUW40_5, QKUW40_90, QKUW40_97, QKUW40_98, QKUW40_99) ~ 1
#Semilla aleatoria para reproducibilidad
seed<-2125
#Opciones de la función
verbose<- F # Seguir los resultados. TRUE= ver proceso de convergencia
init <- 1e2 # puntos iniciales sobre los cuales encontrar una solución global. sugiero 5e2
testiter <- 5e1 # Iteraciones para encontrar convergencia sugiero 5e2
#Otros parámetros son : eps = 1e-6 (tolerancia)

old <- Sys.time()

lca02 <- glca(f_lca, data = arabebarometro_selected[,paste0("QKUW40_", c(1:5, 90, 97, 98, 99))], nclass= 2, seed= seed, verbose= verbose, n.init= init, decreasing=T, maxiter=1e4, testiter = testiter)
lca03 <- glca(f_lca, data = arabebarometro_selected[,paste0("QKUW40_", c(1:5, 90, 97, 98, 99))], nclass= 3, seed= seed, verbose= verbose, n.init= init, decreasing=T, maxiter=1e4, testiter = testiter)
lca04 <- glca(f_lca, data = arabebarometro_selected[,paste0("QKUW40_", c(1:5, 90, 97, 98, 99))], nclass= 4, seed= seed, verbose= verbose, n.init= init, decreasing=T, maxiter=1e4, testiter = testiter)
lca05 <- glca(f_lca, data = arabebarometro_selected[,paste0("QKUW40_", c(1:5, 90, 97, 98, 99))], nclass= 5, seed= seed, verbose= verbose, n.init= init, decreasing=T, maxiter=1e4, testiter = testiter)
lca06 <- glca(f_lca, data = arabebarometro_selected[,paste0("QKUW40_", c(1:5, 90, 97, 98, 99))], nclass= 6, seed= seed, verbose= verbose, n.init= init, decreasing=T, maxiter=1e4, testiter = testiter)
lca07 <- glca(f_lca, data = arabebarometro_selected[,paste0("QKUW40_", c(1:5, 90, 97, 98, 99))], nclass= 7, seed= seed, verbose= verbose, n.init= init, decreasing=T, maxiter=1e4, testiter = testiter)
lca08 <- glca(f_lca, data = arabebarometro_selected[,paste0("QKUW40_", c(1:5, 90, 97, 98, 99))], nclass= 8, seed= seed, verbose= verbose, n.init= init, decreasing=T, maxiter=1e4, testiter = testiter)
lca09 <- glca(f_lca, data = arabebarometro_selected[,paste0("QKUW40_", c(1:5, 90, 97, 98, 99))], nclass= 9, seed= seed, verbose= verbose, n.init= init, decreasing=T, maxiter=1e4, testiter = testiter)
lca10 <- glca(f_lca, data = arabebarometro_selected[,paste0("QKUW40_", c(1:5, 90, 97, 98, 99))], nclass= 10, seed= seed, verbose= verbose, n.init= init, decreasing=T, maxiter=1e4, testiter = testiter)

new_med<-(Sys.time())
paste0("El modelo tomó ",round(new_med-old,2),"minutos hasta que todos los análisis fueron computados")

Obtenemos índices de ajuste y comparamos las soluciones. Para facilidad, definimos sólo 10 iteraciones, pero debiesen utilizarse al menos 250 como referencia para ver la estabilidad de los índices de ajuste.

In [25]:

#para obtener otros índices de ajuste
gof<-
gofglca(lca02, lca03, lca04, lca05, lca06, lca07, lca08, lca09, lca10, test = "chisq")

#número total de clases latentes postuladas
max_classes<- nrow(gof$gtable)+1

#para obtener índice BLRT= Evaluar si el modelo k presenta mejor ajuste que el k-1

#número de iteraciones para bootstrap, se recomienda 5e2
#demora 7 minutos en Google Colab, RAM ampliada // ~10–20 minutos con Google colab normal
nboot= 200
bootlrt<- gofglca(lca02, lca03, lca04, lca05, lca06, lca07, lca08, lca09, lca10, test = "boot", nboot= nboot, seed=2125)

manualcolors <- rev(c('gray20', 'gray50', 'gray80')) # Tres colores en escala de grises
manual_linetypes <- rev(c('solid', 'dashed', 'dotted')) # Tres estilos de línea
levels <- c("logLik", "Gsq", "AIC", "CAIC", "BIC", "entropy", "Res.Df")
labels <- c('Log-Verosimilitud', 'Chi2', 'Criterio de Información\nde Akaike (AIC)',
            'AIC Corregido', 'Criterio de Información\nBayesiano (BIC)', 'Entropía',
            'Grados de libertad residuales')

fig_lca_fit <- cbind.data.frame(rn = 2:max_classes, gof$gtable) %>%
  data.frame() %>%
  dplyr::mutate_if(is.character, as.numeric) %>%  # Convertir columnas de carácter a numérico
  #formatear la base de manera larga (es decir), cada fila es la combinación única de una probabilidad, pregunta y clase
  tidyr::pivot_longer(cols = -rn, names_to = "indices", values_to = "value", values_drop_na = F) %>%
  #dejar en formato factor los índices, para reconocerlos  e identificarlos
  dplyr::mutate(indices = factor(indices, levels = levels, labels = labels)) %>%
  # Filtrar solo AIC y BIC
  dplyr::filter(grepl("(AIC|BIC)", indices, ignore.case = TRUE)) %>%
  dplyr::mutate(ModelIndex = factor(rn, levels = 2:max_classes)) %>%
  ggplot(aes(x = ModelIndex, y = value, group = indices,
             color = indices, linetype = indices)) +
  geom_line(linewidth = 1.5) +
  # Colores y estilos de línea personalizados
  scale_color_manual(values = manualcolors) +
  scale_linetype_manual(values = manual_linetypes) +
  labs(x = "Número de clases", y = "Valor", color = "Medida", linetype = "Medida") +
  theme_bw()

fig_lca_fit

#Para exportar gráfico
ggsave("_fig2_comparison_glca.png",fig_lca_fit, dpi=600)


ERROR: Error in if (maxdiff < eps) {: missing value where TRUE/FALSE needed


Vemos un resumen de la salida del modelo con mejores índices de ajuste BIC.


In [ ]:
summary(
  eval(parse(text = paste0("lca",sprintf("%02.0f",mejor_modelo))))
        ) #

# Analisis


In [ ]:
plot(
     glca_mejor_ajuste
     , ask=F)

Vemos la entropía usando `glca_mejor_ajuste$gof$entropy` `r round(glca_mejor_ajuste$gof$entropy,2)`. Bastante buena.

Ahora veremos los perfiles de las probabilidades de respuesta a los ítems condicionales a la pertenencia a una determinada clase.

In [ ]:
# Configuración inicial y preparación de datos
# Extracción de parámetros rho del modelo GLCA
rho_glca <- do.call("bind_rows", glca_mejor_ajuste$param$rho$ALL) %>%
  t() %>%
  round(2) %>%
  data.table::data.table(keep.rownames = TRUE) %>%
  magrittr::set_colnames(c("variables", paste0("Class", 1:glca_mejor_ajuste$model$C))) %>%
  tidyr::separate(variables, into = c("var", "prob"), sep = ".Y =")

# Transformación de datos del modelo
lcmodel_glca <- rho_glca %>%
  tidyr::pivot_longer(
    cols = 3:5,
    names_to = "class",
    values_to = "value"
  )

# Unión de etiquetas con probabilidades del modelo
lcmodel_glca <- lcmodel_glca %>%
  dplyr::mutate(pr = as.numeric(gsub("[^0-9.]+", "", prob))-1) %>%
  dplyr::mutate(etiqueta= dplyr::case_when(
    grepl("_1$", var) ~ "01.Boicot\nempresas",
    grepl("_2$", var) ~ "02.Seguir\nnoticias",
    grepl("_3$", var) ~ "03.Actividades\nsolidaridad",
    grepl("_4$", var) ~ "04.Difusión\nsolidaridad",
    grepl("_5$", var) ~ "05.Donaciones\nmonetarias",
    grepl("_90$", var) ~ "90.Otro\n(Especificar)",
    grepl("_97$", var) ~ "97.No he tomado\nninguna acción",
    grepl("_98$", var) ~ "98.No sabe",
    grepl("_99$", var) ~ "99.Se niega\nresponder",
    TRUE ~ NA_character_  # Para manejar valores no coincidentes
  ))

# Generación de etiquetas para gráficos
lcmodel_glca$text_label <- paste0(
  "Etiqueta:", lcmodel_glca$etiqueta,
  "<br>Categoría:", lcmodel_glca$pr,
  "<br>%: ", scales::percent(lcmodel_glca$value)
)
#Ordenamos las opciones por orden de aparición
lcmodel_glca$var <- factor(lcmodel_glca$var, levels = paste0("QKUW40_", c(1:5, 90, 97, 98, 99)))

# Cálculo de porcentajes de clase, >.5 PROBABILIDAD DE PERTENENCIA
glca_pop_perc_05 <- glca_mejor_ajuste$posterior$ALL %>%
  dplyr::mutate_all(~ ifelse(. > .5, 1, 0)) %>%
  dplyr::mutate(final_05 = dplyr::case_when(
    `Class 1` == 1 ~ 1, `Class 2` == 1 ~ 2, `Class 3` == 1 ~ 3
  )) %>%
  janitor::tabyl(final_05) %>%
  dplyr::mutate(
    percent = scales::percent(percent, accuracy = .1),
    class = paste0("Clase", final_05),
    print = paste0(class, "\n(n=", n, ";", percent, ")")
  )
# Cálculo de porcentajes de clase, >.7 PROBABILIDAD DE PERTENENCIA
glca_pop_perc_07 <- glca_mejor_ajuste$posterior$ALL %>%
  dplyr::mutate_all(~ ifelse(. > .7, 1, 0)) %>%
  dplyr::mutate(final_07 = dplyr::case_when(
    `Class 1` == 1 ~ 1, `Class 2` == 1 ~ 2, `Class 3` == 1 ~ 3
  )) %>%
  janitor::tabyl(final_07) %>%
  dplyr::mutate(
    percent = scales::percent(percent, accuracy = .1),
    class = paste0("Clase", final_07),
    print_07 = paste0(class, "[>.7 prob]","\n(n=", n, ";", percent, ")")
  ) %>%
  #sacamos sujetos no clasificados
  dplyr::filter(!is.na(final_07)) %>%
  #seleccionamos columnas de interés para unir y no generar redundancia
  dplyr::select(final_07, class, print_07)

# Unión con datos transformados
lcmodel_glca <- lcmodel_glca %>%
  dplyr::mutate(class=gsub("Class","Clase",class)) %>%
  left_join(glca_pop_perc_05, by = c("class")) %>%
  left_join(glca_pop_perc_07, by = c("class")) %>%
  dplyr::select(-class) %>%
  dplyr::rename("class" = "print")

# Gráfico principal
zp <- ggplot(dplyr::mutate(lcmodel_glca, class=gsub("\n","",class)), aes(x = var, y = value, fill = factor(pr), label = text_label)) +
  geom_bar(stat = "identity", position = "stack") +
  facet_grid(class ~ .) +
  scale_fill_brewer(type = "seq", palette = "Greys", na.value = "white") +
  theme_bw() +
  labs(
    y = "Porcentaje de probabilidad de respuesta",
    x = "",
    fill = "Categorías de\nRespuesta"
  ) +
  theme(
    axis.text.y = element_blank(),
    axis.ticks.y = element_blank(),
    panel.grid.major.y = element_blank(),
    axis.text.x = element_text(angle = 30, hjust = 1)
  ) +
  guides(fill = guide_legend(reverse = TRUE))

# Exportación del gráfico y datos
# ggplotly(zp, tooltip = c("text_label")) %>%
#   layout(xaxis = list(showticklabels = TRUE), height = 600, width = 1000)
ggplotly(zp, tooltip = c("text_label")) %>%
  layout(
    legend = list(
      orientation = "h", # Leyenda horizontal
      x = 0.5,           # Centra horizontalmente
      y = -0.2           # Mueve la leyenda debajo del gráfico
    ),
    xaxis = list(showticklabels = TRUE),
    height = 700, # Ajusta la altura
    width = 700, # Ajusta el ancho
    margin = list(l = 100, r = 50, t = 50, b = 150) # Márgenes para etiquetas y títulos
  )

#EXPORTAMOS EL GRÁFICO NO-INTERACTIVO
ggsave("_fig4_LCA_patrones_respuesta.png", zp, dpi = 600)

Generamos una tabla para ver las probabilidades de respuesta condicional a la pertenencia a una clase latente.

In [ ]:
#Exportar las tablas obtenidas
lcmodel_glca %>%
  #excluimos columnas de procesos intermedios que pueden confundirnos
  dplyr::select(-any_of(c("text_label", "prob", "n", "percent", "final_05", "final_07"))) %>%
  #sacamos información redundante
  #utilizamos doble escape (\\) para sacar símbolos que pueden ser interpretados literalmente
  dplyr::mutate(print_07 = gsub("\\[>\\.7 prob\\]\\n", " ", print_07))%>%
  dplyr::mutate((across(c("etiqueta", "class"),~gsub("\n", " ",.))))%>%
  #renombramos columnas para mayor claridad
  dplyr::rename("Prob. de pertencer a clase [P>.5]"="class", "Prob. de pertencer a clase [P>.7]"="print_07") %>%
  {
  rio::export(.,"_output/variables_probabilidades_respuesta.xlsx")
  rio::export(.,"_output/variables_probabilidades_respuesta.csv")
  knitr::kable(.,"markdown", caption= "Probabilidades de respuesta de cada clase y clasificaciones por según distintos criterios de clasificación por probabilidad posterior de pertenencia", escape=F)
  }


<br>

A partir del gráfico anterior y la tabla, podemos interpretar las clases:

**Clase 1**= Describe a personas que no han tomado ninguna acción concreta, no saben o no responden.

**Clase 2**= Describe a personas que han actuado de forma individual e internalizante.

**Clase 3**= Describe a personas que han actuado de muchas formas, incluidas acciones colectivas, mobilizaciones y difusión.

De ahí uno podría hacer interpretaciones en contraste con teorías de movimientos sociales, *advocacy groups*, etc. Por ejemplo, grupos que reflejan actitudes de desmovilización o desconexión, grupos de acciones individuales, simbólicas o de bajo costo ("participación selectiva" / *free-riding*), y un grupo cuya movilización tiende a ser colectiva, incorporando recursos y redes con mayor nivel de compromiso y participación activa.

<br>

In [ ]:

posterior_glca_05<-
glca_mejor_ajuste$posterior$ALL %>%
    dplyr::mutate_all(~ifelse(.>.5,1,0)) %>%
  dplyr::mutate(final_05=dplyr::case_when(`Class 1`==1~1,`Class 2`==1~2, `Class 3`==1~3))

posterior_glca_07<-
glca_mejor_ajuste$posterior$ALL %>%
    dplyr::mutate_all(~ifelse(.>.7,1,0)) %>%
  dplyr::mutate(final_07=dplyr::case_when(`Class 1`==1~1,`Class 2`==1~2, `Class 3`==1~3))


glca_mejor_ajuste$posterior$ALL %>%
  dplyr::mutate_all(~ ifelse(. > .5, 1, 0)) %>%
  dplyr::mutate(final_05 = dplyr::case_when(
    `Class 1` == 1 ~ 1, `Class 2` == 1 ~ 2, `Class 3` == 1 ~ 3
  )) %>%
  dplyr::mutate(final_07= posterior_glca_07$final_07) %>%
  janitor::tabyl(final_05, final_07) %>%
  dplyr::rename("No clasificado" = "NA_") %>%
  knitr::kable(
    col.names = c("Clases", "Clase 1", "Clase 2", "Clase 3", "No clasificado"),
    caption = "Clasificación por valores superiores a 0,7 en probabilidad vs. 0,5"
  )

posterior_glca_07 %>%
    rowwise() %>%
  dplyr::mutate(count_ones = sum(c_across(starts_with("Class")) == 1)) %>%
  ungroup() %>%
  janitor::tabyl(final_07,count_ones) %>%
  knitr::kable(col.names= c("Clasificación por valores superiores a 0,7 en probabilidad", "0", "1"), caption="Pruebas por probabilidades de clasificación posterior")

Vemos que hay algunas observaciones que si clasificáramos por un criterio más estricto de pertenencia (al menos 49% de la variabilidad es explicada por la clase) no alcanzan a pertenecer a una determinada clase.

<br>

# Caracterización/ Comparación con otras variables

## Clasificar-analizar

In [ ]:


invisible("Ocupación (Q1005)")
cbind.data.frame(ocupacion=arabebarometro_selected$Q1005, clasif_05= posterior_glca_05$final_05) %>%
    dplyr::mutate(ocupacion= dplyr::case_when(ocupacion%in%c("1","2")~"activa", ocupacion=="5"~"estudiante", T~"inactiva"))%>%
    dplyr::group_by(ocupacion, clasif_05)%>%
    dplyr::count() %>%
    dplyr::mutate(clasif_05= dplyr::case_when(clasif_05=="1"~"Clase1",clasif_05=="2"~"Clase2", clasif_05=="3"~"Clase3", T~NA_character_))%>%
    tidyr::pivot_wider(names_from = clasif_05, values_from= n)%>%
  dplyr::mutate(total= Clase1+Clase2+Clase3)%>%
  dplyr::mutate(Clase1= glue::glue("{Clase1}({scales::percent(Clase1/total)})"))%>%
  dplyr::mutate(Clase2= glue::glue("{Clase2}({scales::percent(Clase2/total)})"))%>%
  dplyr::mutate(Clase3= glue::glue("{Clase3}({scales::percent(Clase3/total)})"))

cbind.data.frame(ocupacion= arabebarometro_selected$Q1005, clasif_05= posterior_glca_05$final_05) %>%
    dplyr::mutate(ocupacion= dplyr::case_when(ocupacion%in%c("1","2")~"activa", ocupacion=="5"~"estudiante", T~"inactiva"))%>%
    janitor::tabyl(ocupacion, clasif_05)%>%
    janitor::chisq.test()

#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_
#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_#_

invisible("Sexo (Q1002)")
cbind.data.frame(sexo=arabebarometro_selected$Q1002, clasif_05= posterior_glca_05$final_05) %>%
  dplyr::group_by(sexo, clasif_05)%>%
  dplyr::count() %>%
  dplyr::mutate(sexo= ifelse(sexo=="1","hombre", "mujer"))%>%
    dplyr::mutate(clasif_05= dplyr::case_when(clasif_05=="1"~"Clase1",clasif_05=="2"~"Clase2", clasif_05=="3"~"Clase3", T~NA_character_))%>%
  tidyr::pivot_wider(names_from = clasif_05, values_from= n)%>%
  dplyr::mutate(total= Clase1+Clase2+Clase3)%>%
  dplyr::mutate(Clase1= glue::glue("{Clase1}({scales::percent(Clase1/total)})"))%>%
  dplyr::mutate(Clase2= glue::glue("{Clase2}({scales::percent(Clase2/total)})"))%>%
  dplyr::mutate(Clase3= glue::glue("{Clase3}({scales::percent(Clase3/total)})"))

chisq.test(arabebarometro_selected$Q1002, posterior_glca_05$final_05)

chisq.posthoc.test(table(arabebarometro_selected$Q1002,posterior_glca_05$final_05))

cbind.data.frame(sexo=arabebarometro_selected$Q1002, clasif_07= posterior_glca_07$final_07) %>%
  dplyr::group_by(sexo, clasif_07) %>%
  dplyr::count() %>%
  dplyr::mutate(sexo= ifelse(sexo=="1","hombre", "mujer"))%>%
  dplyr::mutate(clasif_07= dplyr::case_when(clasif_07=="1"~"Clase1",clasif_07=="2"~"Clase2", clasif_07=="3"~"Clase3", T~NA_character_))%>%
  tidyr::pivot_wider(names_from = clasif_07, values_from= n)%>%
  dplyr::mutate(total= Clase1+Clase2+Clase3)%>%
  dplyr::mutate(Clase1= glue::glue("{Clase1}({scales::percent(Clase1/total)})"))%>%
  dplyr::mutate(Clase2= glue::glue("{Clase2}({scales::percent(Clase2/total)})"))%>%
  dplyr::mutate(Clase3= glue::glue("{Clase3}({scales::percent(Clase3/total)})"))

#excluimos los casos perdidos
chisq.test(table(arabebarometro_selected$Q1002[!is.na(posterior_glca_07$final_07)],posterior_glca_07$final_07[!is.na(posterior_glca_07$final_07)]))

#obtenemos los resultados
chisq.posthoc.test(table(arabebarometro_selected$Q1002[!is.na(posterior_glca_07$final_07)],posterior_glca_07$final_07[!is.na(posterior_glca_07$final_07)]))

## Modelo con covariable


In [ ]:
f_lca_adj<- item(QKUW40_1, QKUW40_2, QKUW40_3, QKUW40_4, QKUW40_5, QKUW40_90, QKUW40_97, QKUW40_98, QKUW40_99) ~ Q1002 + Q1005
lca03_adj <- glca(f_lca_adj, data = arabebarometro_selected[,c(paste0("QKUW40_", c(1:5, 90, 97, 98, 99)),"Q1002", "Q1005")]%>% dplyr::mutate(Q1005= dplyr::case_when(Q1005%in%c("1","2")~"activa", Q1005=="5"~"estudiante", T~"inactiva")),
    nclass= 3,
    seed= seed,
    verbose= verbose,
    n.init= init,
    decreasing=T,
    maxiter=1e4,
    testiter = testiter)

Vemos los coeficientes de regresión

In [ ]:
lca03_adj$coefficient$ALL



- Podría decirse que hombres empleados (intercepto) tienen mayores chances de pertenecer a la clase 1 vs. la 3  (OR= 11.3), y 2 vs. 3 (OR= 27.7)
- Encontrarse inactivo (retirado, dueñ@ de casa o desempleado y buscando trabajo vs. empleados) se asocia a menores chances de pertenecer a la clase 1 vs. la 3.
- No hay evidencia de una asociación entre pertenecer a una clase y el sexo.

Tener en cuenta la escasez de observaciones para algunas categorías de respuesta.

In [ ]:
# Crear un data frame combinado con ocupación, clasificación y sexo
data_combined <- cbind.data.frame(
    ocupacion = arabebarometro_selected$Q1005,
    clasif_05 = posterior_glca_05$final_05,
    sexo = arabebarometro_selected$Q1002
) %>%
    # Reemplazar valores en la variable 'ocupacion'
    mutate(
        ocupacion = case_when(
            ocupacion %in% c("1", "2") ~ "activa",
            ocupacion == "5" ~ "estudiante",
            TRUE ~ "inactiva"
        )
    )

# Crear tabla de frecuencias cruzadas con xtabs
tabla_cruzada <- xtabs(~ clasif_05 + ocupacion + sexo, data = data_combined)

# Información de la sesión


In [ ]:
#guardamos la información generada
save.image("palestine2_25.RData")

#vemos desde donde se producen las librerías
message(paste0("R library: ", Sys.getenv("R_LIBS_USER")))
message(paste0("Date: ",withr::with_locale(new = c('LC_TIME' = 'C'), code =Sys.time())))
message(paste0("Editor context: ", getwd()))

#plataforma donde se desplegó el código
devtools::session_info()$platform

In [ ]:

sesion_info <- devtools::session_info()
dplyr::select(
  tibble::as_tibble(sesion_info$packages),
  c(package, loadedversion, source)
) %>%
 knitr::kable("markdown", caption = "Información paquetes de R", format = "html",
      col.names = c("Paquete", "Versión", "Fuente"),
    row.names = FALSE,
      align = c("c", "l", "r"))